In [19]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base-mlm")  # or a code search variant

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [20]:
# Example code snippet and documentation/comment
code = "def add(a, b): return a + b"
documentation = "Function to add two numbers"

# Tokenize code
code_tokens = tokenizer.encode(code, return_tensors='pt')
# Tokenize natural language comment
doc_tokens = tokenizer.encode(documentation, return_tensors='pt')

In [21]:
import torch
# Get embeddings for code
with torch.no_grad():
    code_embeddings = model(code_tokens)[0]

# Get embeddings for documentation/comment
with torch.no_grad():
    doc_embeddings = model(doc_tokens)[0]

In [22]:
import faiss
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaModel

# loading CodeBERT
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

# demo code snippets
code_snippets = [
    "def add(a, b): return a + b"
]

code_embedding_list = []

for code_snippet in code_snippets:
    # tokenzing the code snippet
    code_tokens = tokenizer.encode(code_snippet, return_tensors='pt')
    with torch.no_grad():
        code_outputs = model(code_tokens)
        code_embeddings = code_outputs.last_hidden_state
        code_embedding = torch.mean(code_embeddings, dim=1).squeeze(0)
    code_embedding_list.append(code_embedding.numpy())

code_embedding_array = np.array(code_embedding_list)  # Shape: (n_samples, hidden_size)

# creating faiss index
d = code_embedding_array.shape[1]
index = faiss.IndexFlatL2(d)
index.add(code_embedding_array.astype('float32'))


In [23]:
# query
query = "Function to add two numbers"

# Tokenize and get embedding
query_tokens = tokenizer.encode(query, return_tensors='pt')
with torch.no_grad():
    query_outputs = model(query_tokens)
    query_embeddings = query_outputs.last_hidden_state
    query_embedding = torch.mean(query_embeddings, dim=1).squeeze(0).numpy()

# Ensure the embedding is float32
query_embedding = np.array([query_embedding.astype('float32')])  # Shape: (1, d)

# Perform the search
k = 5  # Number of nearest neighbors
distances, indices = index.search(query_embedding, k)

# Retrieve the corresponding code snippets
for idx in indices[0]:
    print(f"Matched Code Snippet: {code_snippets[idx]}")

Matched Code Snippet: def add(a, b): return a + b
Matched Code Snippet: def add(a, b): return a + b
Matched Code Snippet: def add(a, b): return a + b
Matched Code Snippet: def add(a, b): return a + b
Matched Code Snippet: def add(a, b): return a + b
